# Code Generator — Week 4 Day 4 Challenge

**Igniters-Tunde-Wey-Week-4** (rewritten from week4-day4-challenge)

Use an open-source or frontier model to generate high-performance C++ code from Python (or add comments, or write unit tests). Optionally use a Hugging Face endpoint for an open-source model.

**Important:** If you use Hugging Face endpoints, pause them when not in use to avoid cost. [HuggingFace Endpoints UI](https://ui.endpoints.huggingface.co/). In Week 8 we use Modal (pay per use, free credits).

In [ ]:
import os
import io
import sys
import subprocess
import platform
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr

load_dotenv(override=True)
os.environ.setdefault("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY", ""))
os.environ.setdefault("ANTHROPIC_API_KEY", os.getenv("ANTHROPIC_API_KEY", ""))
os.environ.setdefault("HF_TOKEN", os.getenv("HF_TOKEN", ""))

In [ ]:
openai_client = OpenAI()
claude_client = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [ ]:
PROMPT_OPTIONS = ["Convert to C", "Add comments", "Write unit tests"]

SYSTEM_PROMPTS = {
    PROMPT_OPTIONS[0]: """
You are an assistant that reimplements Python code in high performance C++.
Respond only with C++ code; use comments sparingly. Do not provide any explanation other than occasional comments.
The C++ must produce identical output in the fastest possible time. Keep RNG implementations identical so results match.
""",
    PROMPT_OPTIONS[1]: """
You are an assistant that adds succinct comments and docstrings to Python code. Respond only with valid Python code.
""",
    PROMPT_OPTIONS[2]: """
You are an assistant that creates unit tests for Python code. Respond only with valid Python code.
""",
}

USER_PROMPTS = {
    PROMPT_OPTIONS[0]: """
Rewrite this Python code in C++ with the fastest possible implementation that produces identical output.
Respond only with C++ code; do not explain. Pay attention to number types (no int overflows).
Include all necessary headers (e.g. #include <iomanip>).

""",
    PROMPT_OPTIONS[1]: "Keep this Python code but insert appropriate comments and docstrings.",
    PROMPT_OPTIONS[2]: "Create unit tests for this Python code.",
}

In [ ]:
SAMPLE_OPTIONS = ["Hello, World", "Calculate pi", "Kadane's Algorithm", "Sieve of Eratosthenes"]

PYTHON_SAMPLES = {
    SAMPLE_OPTIONS[0]: """
import time
start_time = time.time()
print("Hello, world")
end_time = time.time()
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
""",
    SAMPLE_OPTIONS[1]: """
import time
def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result
start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()
print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
""",
    SAMPLE_OPTIONS[2]: """
# Be careful to support large number sizes
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum
def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum
n, initial_seed, min_val, max_val = 10000, 42, -10, 10
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()
print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
""",
    SAMPLE_OPTIONS[3]: """
import time
start_time = time.time()
stop_at = 100_000_000
prime = [True] * (stop_at + 1)
p = 2
while p * p <= stop_at:
    if prime[p]:
        for i in range(p * p, stop_at + 1, p):
            prime[i] = False
    p += 1
primes = [p for p in range(2, stop_at + 1) if prime[p]]
end_time = time.time()
print("Maximum prime: {:,}".format(primes[-1]))
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
""",
}

In [ ]:
def build_user_prompt(instruction: str, python_code: str) -> str:
    return instruction.strip() + "\n" + python_code

def build_openai_messages(system_prompt: str, user_prompt: str):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

In [ ]:
def write_cpp_file(filename_base: str, cpp_code: str) -> None:
    code_to_write = cpp_code.replace("```cpp", "").replace("```", "").strip()
    with open(f"{filename_base}.cpp", "w") as f:
        f.write(code_to_write)

In [ ]:
MODEL_OPTIONS = ["GPT", "Claude"]
DEFAULT_MODEL = MODEL_OPTIONS[0]

def _strip_code_blocks(text: str) -> str:
    return text.replace("```cpp\n", "").replace("```", "")

def stream_gpt(system_prompt, user_prompt, python_code):
    full_prompt = build_user_prompt(user_prompt, python_code)
    messages = build_openai_messages(system_prompt, full_prompt)
    stream = openai_client.chat.completions.create(
        model=OPENAI_MODEL, messages=messages, stream=True
    )
    reply = ""
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ""
        yield _strip_code_blocks(reply)

def stream_claude(system_prompt, user_prompt, python_code):
    full_prompt = build_user_prompt(user_prompt, python_code)
    result = claude_client.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_prompt,
        messages=[{"role": "user", "content": full_prompt}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield _strip_code_blocks(reply)

def call_llm(system_prompt, user_prompt, python_code, model_name):
    if model_name == "GPT":
        gen = stream_gpt(system_prompt, user_prompt, python_code)
    elif model_name == "Claude":
        gen = stream_claude(system_prompt, user_prompt, python_code)
    else:
        raise ValueError(f"Unknown model: {model_name}")
    for chunk in gen:
        yield chunk

In [ ]:
def execute_python(code: str) -> str:
    try:
        buf = io.StringIO()
        sys.stdout = buf
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return buf.getvalue()

def execute_cpp(cpp_code: str, compiler_info) -> str:
    write_cpp_file("optimized", cpp_code)
    if not compiler_info or not compiler_info[2]:
        return "No compiler available."
    compile_cmd = compiler_info[2]
    try:
        subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        run_cmd = ["optimized.exe"] if platform.system() == "Windows" else ["./optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error:\n{e.stderr or e.stdout}"

In [ ]:
VISUAL_STUDIO_2022 = "C:\\Program Files\\Microsoft Visual Studio\\2022\\Community\\Common7\\Tools\\VsDevCmd.bat"
VISUAL_STUDIO_2019 = "C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\BuildTools\\Common7\\Tools\\VsDevCmd.bat"
SIMPLE_CPP = '''
#include <iostream>
int main() { std::cout << "Hello"; return 0; }
'''

def _run_cmd(cmd):
    try:
        r = subprocess.run(cmd, check=True, text=True, capture_output=True)
        return r.stdout if r.stdout else "SUCCESS"
    except Exception:
        return ""

def get_compiler_info(filename_base: str):
    plat = platform.system()
    result = []
    try:
        with open("simple.cpp", "w") as f:
            f.write(SIMPLE_CPP)
        if plat == "Windows":
            for vs_name, vs_path in [("Visual Studio 2022", VISUAL_STUDIO_2022), ("Visual Studio 2019", VISUAL_STUDIO_2019)]:
                if os.path.isfile(vs_path):
                    for f in ["simple.exe", "simple.obj"]:
                        if os.path.isfile(f):
                            os.remove(f)
                    cmd = ["cmd", "/c", f"call \"{vs_path}\" && cl simple.cpp"]
                    if _run_cmd(cmd):
                        out = _run_cmd(["simple.exe"])
                        if out and "Hello" in out:
                            result = ["Windows", vs_name, ["cmd", "/c", f"call \"{vs_path}\" && cl {filename_base}.cpp"]]
                            break
            if not result:
                result = [plat, "Unavailable", []]
        elif plat == "Linux":
            for compiler, args in [("GCC (g++)", ["g++", "simple.cpp", "-o", "simple"]), ("Clang++", ["clang++", "simple.cpp", "-o", "simple"])]:
                if os.path.isfile("./simple"):
                    os.remove("./simple")
                if _run_cmd(args):
                    if _run_cmd(["./simple"]) == "Hello":
                        result = ["Linux", compiler, [args[0], f"{filename_base}.cpp", "-o", filename_base]]
                        break
            if not result:
                result = [plat, "Unavailable", []]
        elif plat == "Darwin":
            if os.path.isfile("./simple"):
                os.remove("./simple")
            args = ["clang++", "-Ofast", "-std=c++17", "-o", "simple", "simple.cpp"]
            if _run_cmd(args) and _run_cmd(["./simple"]) == "Hello":
                result = ["Macintosh", "Clang++", ["clang++", "-Ofast", "-std=c++17", "-o", filename_base, f"{filename_base}.cpp"]]
            if not result:
                result = [plat, "Unavailable", []]
        else:
            result = [plat, "Unavailable", []]
    except Exception:
        result = [plat or "Unknown", "Unavailable", []]
    return result if result else ["Unknown", "Unavailable", []]

In [ ]:
COMPILER_INFO = get_compiler_info("optimized")
selected_tab_key = PROMPT_OPTIONS[0]

def on_tab_select(evt: gr.SelectData):
    global selected_tab_key
    selected_tab_key = evt.value

def reset_prompts():
    return SYSTEM_PROMPTS[selected_tab_key], USER_PROMPTS[selected_tab_key]

def on_sample_change(sample_name, current_code):
    if sample_name != "Custom" and sample_name in PYTHON_SAMPLES:
        return PYTHON_SAMPLES[sample_name]
    return current_code

def on_code_edit():
    return "Custom"

def run_cpp(cpp_code):
    return execute_cpp(cpp_code, COMPILER_INFO)

CSS = ".python {background-color: #306998;} .cpp {background-color: #050;}"

with gr.Blocks(css=CSS) as ui:
    with gr.Tab(PROMPT_OPTIONS[0]) as tab1:
        gr.Markdown("# " + PROMPT_OPTIONS[0])
        with gr.Group():
            with gr.Row():
                sys1 = gr.Textbox(label="System prompt", value=SYSTEM_PROMPTS[PROMPT_OPTIONS[0]], lines=10, interactive=True)
                usr1 = gr.Textbox(label="User prompt", value=USER_PROMPTS[PROMPT_OPTIONS[0]], lines=10, interactive=True)
            gr.Button("Reset prompts").click(reset_prompts, outputs=[sys1, usr1])
        with gr.Row():
            with gr.Column():
                rad1 = gr.Radio(SAMPLE_OPTIONS + ["Custom"], label="Sample program", value=SAMPLE_OPTIONS[0])
                py1 = gr.Textbox(label="Python code", value=PYTHON_SAMPLES[SAMPLE_OPTIONS[0]], lines=10, interactive=True)
            with gr.Column():
                model1 = gr.Dropdown(MODEL_OPTIONS, label="Model", value=DEFAULT_MODEL)
                cpp1 = gr.Textbox(label="C++ code", lines=10, interactive=True)
                gr.Button("Convert code").click(call_llm, inputs=[sys1, usr1, py1, model1], outputs=[cpp1])
        with gr.Row():
            with gr.Column():
                out_py1 = gr.TextArea(label="Python result", elem_classes=["python"])
                gr.Button("Run Python").click(execute_python, inputs=[py1], outputs=[out_py1])
            with gr.Column():
                out_cpp1 = gr.TextArea(label="C++ result", elem_classes=["cpp"])
                btn_cpp = gr.Button("Run C++" if COMPILER_INFO[1] != "Unavailable" else "No compiler", interactive=(COMPILER_INFO[1] != "Unavailable"))
                btn_cpp.click(run_cpp, inputs=[cpp1], outputs=[out_cpp1])
                gr.Radio([COMPILER_INFO[0]], label="Architecture", value=COMPILER_INFO[0], interactive=False)
                gr.Radio([COMPILER_INFO[1]], label="Compiler", value=COMPILER_INFO[1], interactive=False)
        rad1.change(on_sample_change, [rad1, py1], [py1])
        py1.change(on_code_edit, outputs=[rad1])
    tab1.select(on_tab_select)

    with gr.Tab(PROMPT_OPTIONS[1]) as tab2:
        gr.Markdown("# " + PROMPT_OPTIONS[1])
        with gr.Group():
            with gr.Row():
                sys2 = gr.Textbox(label="System prompt", value=SYSTEM_PROMPTS[PROMPT_OPTIONS[1]], lines=10, interactive=True)
                usr2 = gr.Textbox(label="User prompt", value=USER_PROMPTS[PROMPT_OPTIONS[1]], lines=10, interactive=True)
            gr.Button("Reset prompts").click(reset_prompts, outputs=[sys2, usr2])
        with gr.Row():
            with gr.Column():
                rad2 = gr.Radio(SAMPLE_OPTIONS + ["Custom"], label="Sample program", value=SAMPLE_OPTIONS[1])
                py2 = gr.Textbox(label="Python code", value=PYTHON_SAMPLES[SAMPLE_OPTIONS[1]], lines=10, interactive=True)
            with gr.Column():
                model2 = gr.Dropdown(MODEL_OPTIONS, label="Model", value=DEFAULT_MODEL)
                gr.Button("Comment code").click(call_llm, inputs=[sys2, usr2, py2, model2], outputs=[com2 := gr.Textbox(label="Commented code", lines=20)])
        rad2.change(on_sample_change, [rad2, py2], [py2])
        py2.change(on_code_edit, outputs=[rad2])
    tab2.select(on_tab_select)

    with gr.Tab(PROMPT_OPTIONS[2]) as tab3:
        gr.Markdown("# " + PROMPT_OPTIONS[2])
        with gr.Group():
            with gr.Row():
                sys3 = gr.Textbox(label="System prompt", value=SYSTEM_PROMPTS[PROMPT_OPTIONS[2]], lines=10, interactive=True)
                usr3 = gr.Textbox(label="User prompt", value=USER_PROMPTS[PROMPT_OPTIONS[2]], lines=10, interactive=True)
            gr.Button("Reset prompts").click(reset_prompts, outputs=[sys3, usr3])
        with gr.Row():
            with gr.Column():
                rad3 = gr.Radio(SAMPLE_OPTIONS + ["Custom"], label="Sample program", value=SAMPLE_OPTIONS[1])
                py3 = gr.Textbox(label="Python code", value=PYTHON_SAMPLES[SAMPLE_OPTIONS[1]], lines=10, interactive=True)
            with gr.Column():
                model3 = gr.Dropdown(MODEL_OPTIONS, label="Model", value=DEFAULT_MODEL)
                gr.Button("Create unit tests").click(call_llm, inputs=[sys3, usr3, py3, model3], outputs=[ut3 := gr.Textbox(label="Unit tests", lines=20)])
        rad3.change(on_sample_change, [rad3, py3], [py3])
        py3.change(on_code_edit, outputs=[rad3])
    tab3.select(on_tab_select)

ui.launch(inbrowser=True)